In [1]:
import geopandas as gpd

In [4]:
punkter = gpd.read_parquet(f"C:/Users/ort/OneDrive - Statistisk sentralbyrå/data/tilfeldige_adresser_1000.parquet")

Hvis man vil lage egne geopandas-funksjoner, for eksempel en skreddersydd dissolve:

In [3]:
def diss(gdf, **qwargs):

    kopi = gdf.copy()
    kopi = kopi.loc[:, ~kopi.columns.str.contains('index|level_')]
    
    dissolvet = kopi.dissolve(**qwargs)

    dissolvet["geometry"] = dissolvet.geometry.apply(lambda x: x.buffer(0) if x.buffer(0).is_empty==False else x) #buffer(0) gir EMPTY POLYGON for punkter og linjer
    
    dissolvet = dissolvet.reset_index()
    
    dissolvet.columns = ["_".join(kolonne).strip("_") 
                         if isinstance(kolonne, tuple) else kolonne 
                         for kolonne in dissolvet.columns]

    return dissolvet.loc[:, ~dissolvet.columns.str.contains('index|level_')]

Kan denne utføres sånn:

In [4]:
dissolvet = diss(punkter)
dissolvet

,geometry,idx
0,"MULTIPOINT (254575.200 6661631.500, 255727.900...",1


Alternativt kan man legge funksjonen til i pandas.DataFrame-classen sånn her:

In [5]:
from pandas.core.base import PandasObject
PandasObject.diss = diss

Da kan man skrive det sånn:

In [6]:
dissolvet = punkter.diss()
dissolvet

,geometry,idx
0,"MULTIPOINT (254575.200 6661631.500, 255727.900...",1


Som gjør det lettere å chaine flere funksjoner etter hverandre:

In [7]:
import sys
sys.path.append(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\Dokumenter\GitHub")
import geopandasgreier as gg

dissolvet = (punkter
             .buff(10)
             .buff(10)
             .buffdissexp(-10)
)

En ulempe er at man gjør seg avhengig av at dette funker: 

In [8]:
PandasObject.diss = diss

Heldigvis kan man også gjøre dette med ren Python ved å lage en egen subclass av geopandas.GeoDataFrame:

In [2]:
class MinGeoDataFrame(gpd.GeoDataFrame):
    
    def buff(self, avstand, resolution=50, **qwargs):
        kopi = self.copy()
        kopi["geometry"] = kopi.buffer(avstand, resolution=resolution, **qwargs).buffer(0)
        return MinGeoDataFrame(kopi)
    
    def diss(self, **qwargs):
        kopi = self.copy()
        kopi = kopi.loc[:, ~kopi.columns.str.contains('index|level_')]
        
        dissolvet = kopi.dissolve(**qwargs)

        dissolvet["geometry"] = dissolvet.geometry.apply(lambda x: x.buffer(0) if x.buffer(0).is_empty==False else x) #buffer(0) gir EMPTY POLYGON for punkter og linjer
        
        dissolvet = dissolvet.reset_index()
        
        dissolvet.columns = ["_".join(kolonne).strip("_") 
                            if isinstance(kolonne, tuple) else kolonne 
                            for kolonne in dissolvet.columns]

        dissolvet = dissolvet.loc[:, ~dissolvet.columns.str.contains('index|level_')]

        return MinGeoDataFrame(dissolvet)

Da må man legge til denne linja hver gang man leser en fil som en GeoDataFrame:

In [5]:
punkter = MinGeoDataFrame(punkter)

In [7]:
dissolvet = (punkter
             .buff(10)
             .diss()
             .dissolve()
)
dissolvet

,geometry,idx
0,"MULTIPOLYGON (((258089.471 6648494.779, 258089...",1


Og punktene er ikke overskrevet og bufret:

In [ ]:
punkter